In [1]:
import time
#time.sleep(60*60*2.5)

In [2]:
import numpy as np
import os
import pickle
import math

import os
import sys
import io
import numpy as np
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline


from collections import defaultdict

In [3]:
import math

def angle_to_sin_cos(angle):
    return(math.sin(angle), math.cos(angle))

def flatten_frame(frame, force_power, force_angle):
    result = list()
    for element in frame:
        result += element[0]
        for axis in range(3):
            result += angle_to_sin_cos(element[0][axis])
    result.append(force_power)
    result += angle_to_sin_cos(force_angle)
    return result


In [5]:
import pickle
import csv

x_delta = list()
x_abs_0 = list()
x_abs = list()
x_force = list()
y_abs_out = list()
y_delta_out = list()

def write_sample(data, name):
    print("writing: ", name)
    with open(F'samplesGeneration/demo/{name}.csv', 'w', newline='\n') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for num, row in enumerate(data):            
            if not num % 10:
                print(num, "/", len(data), end="\r")
            writer.writerow(data)

def process_sample(sample, sample_name, write_to_csv):
    force_power, force_angle = [float(x) for x in sample_name.split("_")]
    result = list()
    for n in range(1, 239):
        f0_data = list()
        f1_data = list()
        for element in sample[n-1]:
            f0_data.append(element[0] + element[1][:-1])
        for element in sample[n]:
            f1_data.append(element[0] + element[1][:-1])
        
        f0_data = np.array(f0_data)
        f1_data = np.array(f1_data)
        x_delta.append(f1_data - f0_data)
        
        f1_abs = list()
        for element in sample[n]:
            angles = list()
            angles += angle_to_sin_cos(element[1][0])
            angles += angle_to_sin_cos(element[1][1])
            angles += angle_to_sin_cos(element[1][2])
            f1_abs.append(element[0] + angles)
        f1_abs = np.array(f1_abs)
        x_abs.append(f1_abs)
        
        f0_abs = list()
        for element in sample[n - 1]:
            angles = list()
            angles += angle_to_sin_cos(element[1][0])
            angles += angle_to_sin_cos(element[1][1])
            angles += angle_to_sin_cos(element[1][2])
            f0_abs.append(element[0] + angles)
        f0_abs = np.array(f0_abs)
        x_abs_0.append(f0_abs)
        
        y_abs = list()
        for element in sample[n + 1]:
            angles = list()
            angles += angle_to_sin_cos(element[1][0])
            angles += angle_to_sin_cos(element[1][1])
            angles += angle_to_sin_cos(element[1][2])
            y_abs.append(element[0] + angles)
        y_abs = np.array(y_abs)
        y_abs_out.append(y_abs)
        
        y_delta = list()
        for element in sample[n + 1]:
            y_delta.append(element[0] + element[1][:-1])
        y_delta = np.array(y_delta)
        y_delta -= f1_data       
        y_delta_out.append(y_delta)
        
        force_sin, force_cos = angle_to_sin_cos(force_angle)
        x_force.append([force_power, force_sin, force_cos])
    
    if write_to_csv:
        write_sample(np.array(x_abs).reshape((-1,)), "x_abs")
        write_sample(np.array(x_delta).reshape((-1,)), "x_delta")
        write_sample(np.array(x_force).reshape((-1,)), "x_force")
        write_sample(np.array(y_abs_out).reshape((-1,)), "y_abs_out")
        write_sample(np.array(y_delta_out).reshape((-1,)), "y_delta_out")

data_dir = os.path.join(os.getcwd(), "samplesGeneration", "data")
models_dir = os.path.join(os.getcwd(), "model")

data = list()
files = os.listdir(data_dir)
for num, fname in enumerate(files):
    if not num % 10:
        print(num, "/", len(files), end="\r")
    with open(os.path.join(data_dir, fname), "rb") as f:
        data.append(process_sample(pickle.load(f, encoding='latin1'), fname.rsplit(".", 1)[0], num==0))

writing:  x_abs
writing:  x_delta / 8568 / 8568 / 85688568 / 85688568 / 8568856885688568 / 85688568 / 85688568 / 8568 / 8568 / 8568
writing:  x_force2 / 5712 / 5712 / 5712 / 5712 / 57125712 / 5712 / 5712 / 5712 / 5712
writing:  y_abs_out
writing:  y_delta_out8 / 85688568 8568 / 8568 / 85688568 / 8568


In [6]:

x_delta = np.array(x_delta)
x_abs = np.array(x_abs)
x_force = np.array(x_force)
y_abs_out = np.array(y_abs_out)
y_delta_out = np.array(y_delta_out)
x_abs_0 = np.array(x_abs_0)

print("x_delta shape: ", x_delta.shape)
print("x_abs shape: ", x_abs.shape)
print("x_force shape: ", x_force.shape)
print("y_abs_out shape: ", y_abs_out.shape)
print("y_delta_out shape: ", y_delta_out.shape)
print("x_abs_0 shape: ", x_abs_0.shape)

x_delta shape:  (1718360, 4, 6)
x_abs shape:  (1718360, 4, 9)
x_force shape:  (1718360, 3)
y_abs_out shape:  (1718360, 4, 9)
y_delta_out shape:  (1718360, 4, 6)
x_abs_0 shape:  (1718360, 4, 9)


In [7]:
# New way

import sqlite3
import numpy as np

def adapt_array(arr):
    """
    Save Numpy array to SqLite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    """ 
    Load Numpy array from Sqlite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

data_path = os.path.join(os.getcwd(), "samplesGeneration", "data_2021_5.bd")

con = sqlite3.connect(data_path,isolation_level=None)
cur = con.cursor()
cur.execute('PRAGMA synchronous = 0')
cur.execute('PRAGMA journal_mode = OFF') 
cur.execute("create table if not exists data (id INTEGER PRIMARY KEY AUTOINCREMENT, x_delta array, x_abs_0 array, x_abs array, x_force array, y_abs array, y_delta array)")
cur.executemany("insert into data (x_delta, x_abs_0, x_abs, x_force, y_abs, y_delta) values (?,?,?,?,?,?)", zip(x_delta, x_abs_0, x_abs, x_force, y_abs_out, y_delta_out))
con.commit()
con.close()